In [ ]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
DATASET_PATH = '/kaggle/input/tensorflow-great-barrier-reef/train_images/'
CKPT_PATH_1 = '/kaggle/input/cotsy5m/best0.pt'
CKPT_PATH_2 = '/kaggle/input/cotsy5m/best1.pt'
CKPT_PATH_3 = '/kaggle/input/cotsy5m/best2.pt'


IMG_SIZE_1  = 2900
CONF_1      = 0.1
IOU_1       = 0.90

IMG_SIZE_2  = 2900
CONF_2      = 0.1
IOU_2       = 0.90

IMG_SIZE_3  = 2900
CONF_3      = 0.1
IOU_3       = 0.90


DEBUG = True

import numpy as np
import pandas as pd
import sys
import cv2
import torch
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

import os
import torch
import importlib


from PIL import Image
from IPython.display import display

sys.path.append('../input/tensorflow-great-barrier-reef')

In [ ]:
%pwd

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolo5data/Arial.ttf /root/.config/Ultralytics/

In [ ]:
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index

In [ ]:
import warnings
import numpy as np
from numba import jit
import time

@jit(nopython=True)
def bb_intersection_over_union(A, B) -> float:
    xA = max(A[0], B[0])
    yA = max(A[1], B[1])
    xB = min(A[2], B[2])
    yB = min(A[3], B[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    if interArea == 0:
        return 0.0

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (A[2] - A[0]) * (A[3] - A[1])
    boxBArea = (B[2] - B[0]) * (B[3] - B[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou


def prefilter_boxes(boxes, scores, labels, weights, thr):

    new_boxes = dict()

    for t in range(len(boxes)):

        if len(boxes[t]) != len(scores[t]):
            print('Error. Length of boxes arrays not equal to length of scores array: {} != {}'.format(len(boxes[t]), len(scores[t])))
            exit()

        if len(boxes[t]) != len(labels[t]):
            print('Error. Length of boxes arrays not equal to length of labels array: {} != {}'.format(len(boxes[t]), len(labels[t])))
            exit()

        for j in range(len(boxes[t])):
            score = scores[t][j]
            if score < thr:
                continue
            label = int(labels[t][j])
            box_part = boxes[t][j]
            x1 = float(box_part[0])
            y1 = float(box_part[1])
            x2 = float(box_part[2])
            y2 = float(box_part[3])

            # Box data checks
            if x2 < x1:
                warnings.warn('X2 < X1 value in box. Swap them.')
                x1, x2 = x2, x1
            if y2 < y1:
                warnings.warn('Y2 < Y1 value in box. Swap them.')
                y1, y2 = y2, y1
            if x1 < 0:
                warnings.warn('X1 < 0 in box. Set it to 0.')
                x1 = 0
            if x1 > 1:
                warnings.warn('X1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x1 = 1
            if x2 < 0:
                warnings.warn('X2 < 0 in box. Set it to 0.')
                x2 = 0
            if x2 > 1:
                warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x2 = 1
            if y1 < 0:
                warnings.warn('Y1 < 0 in box. Set it to 0.')
                y1 = 0
            if y1 > 1:
                warnings.warn('Y1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y1 = 1
            if y2 < 0:
                warnings.warn('Y2 < 0 in box. Set it to 0.')
                y2 = 0
            if y2 > 1:
                warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y2 = 1
            if (x2 - x1) * (y2 - y1) == 0.0:
                warnings.warn("Zero area box skipped: {}.".format(box_part))
                continue

            # [label, score, weight, model index, x1, y1, x2, y2]
            b = [int(label), float(score) * weights[t], weights[t], t, x1, y1, x2, y2]
            if label not in new_boxes:
                new_boxes[label] = []
            new_boxes[label].append(b)

    # Sort each list in dict by score and transform it to numpy array
    for k in new_boxes:
        current_boxes = np.array(new_boxes[k])
        new_boxes[k] = current_boxes[current_boxes[:, 1].argsort()[::-1]]

    return new_boxes


def get_weighted_box(boxes, conf_type='avg'):
    
    box = np.zeros(8, dtype=np.float32)
    conf = 0
    conf_list = []
    w = 0
    for b in boxes:
        box[4:] += (b[1] * b[4:])
        conf += b[1]
        conf_list.append(b[1])
        w += b[2]
    box[0] = boxes[0][0]
    if conf_type == 'avg':
        box[1] = conf / len(boxes)
    elif conf_type == 'max':
        box[1] = np.array(conf_list).max()
    elif conf_type in ['box_and_model_avg', 'absent_model_aware_avg']:
        box[1] = conf / len(boxes)
    box[2] = w
    box[3] = -1 
    box[4:] /= conf
    return box


def find_matching_box(boxes_list, new_box, match_iou):
    best_iou = match_iou
    best_index = -1
    for i in range(len(boxes_list)):
        box = boxes_list[i]
        if box[0] != new_box[0]:
            continue
        iou = bb_intersection_over_union(box[4:], new_box[4:])
        if iou > best_iou:
            best_index = i
            best_iou = iou

    return best_index, best_iou


def find_matching_box_quickly(boxes_list, new_box, match_iou):
    
    def bb_iou_array(boxes, new_box):
        # bb interesection over union
        xA = np.maximum(boxes[:, 0], new_box[0])
        yA = np.maximum(boxes[:, 1], new_box[1])
        xB = np.minimum(boxes[:, 2], new_box[2])
        yB = np.minimum(boxes[:, 3], new_box[3])

        interArea = np.maximum(xB - xA, 0) * np.maximum(yB - yA, 0)

        # compute the area of both the prediction and ground-truth rectangles
        boxAArea = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
        boxBArea = (new_box[2] - new_box[0]) * (new_box[3] - new_box[1])

        iou = interArea / (boxAArea + boxBArea - interArea)

        return iou

    if boxes_list.shape[0] == 0:
        return -1, match_iou

    # boxes = np.array(boxes_list)
    boxes = boxes_list

    ious = bb_iou_array(boxes[:, 4:], new_box[4:])

    ious[boxes[:, 0] != new_box[0]] = -1

    best_idx = np.argmax(ious)
    best_iou = ious[best_idx]

    if best_iou <= match_iou:
        best_iou = match_iou
        best_idx = -1

    return best_idx, best_iou


def weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=0.55, skip_box_thr=0.0, conf_type='avg', allows_overflow=False):

    if weights is None:
        weights = np.ones(len(boxes_list))
    if len(weights) != len(boxes_list):
        print('Warning: incorrect number of weights {}. Must be: {}. Set weights equal to 1.'.format(len(weights), len(boxes_list)))
        weights = np.ones(len(boxes_list))
    weights = np.array(weights)

    if conf_type not in ['avg', 'max', 'box_and_model_avg', 'absent_model_aware_avg']:
        print('Unknown conf_type: {}. Must be "avg", "max" or "box_and_model_avg", or "absent_model_aware_avg"'.format(conf_type))
        exit()

    filtered_boxes = prefilter_boxes(boxes_list, scores_list, labels_list, weights, skip_box_thr)
    if len(filtered_boxes) == 0:
        return np.zeros((0, 4)), np.zeros((0,)), np.zeros((0,))

    overall_boxes = []
    for label in filtered_boxes:
        boxes = filtered_boxes[label]
        new_boxes = []
        weighted_boxes = np.empty((0,8))
        # Clusterize boxes
        for j in range(0, len(boxes)):
            index, best_iou = find_matching_box_quickly(weighted_boxes, boxes[j], iou_thr)

            if index != -1:
                new_boxes[index].append(boxes[j])
                weighted_boxes[index] = get_weighted_box(new_boxes[index], conf_type)
            else:
                new_boxes.append([boxes[j].copy()])
                weighted_boxes = np.vstack((weighted_boxes, boxes[j].copy()))
        # Rescale confidence based on number of models and boxes
        for i in range(len(new_boxes)):
            clustered_boxes = np.array(new_boxes[i])
            if conf_type == 'box_and_model_avg':
                # weighted average for boxes
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * len(clustered_boxes) / weighted_boxes[i, 2]
                # identify unique model index by model index column
                _, idx = np.unique(clustered_boxes[:, 3], return_index=True)
                # rescale by unique model weights
                weighted_boxes[i, 1] = weighted_boxes[i, 1] *  clustered_boxes[idx, 2].sum() / weights.sum()
            elif conf_type == 'absent_model_aware_avg':
                # get unique model index in the cluster
                models = np.unique(clustered_boxes[:, 3]).astype(int)
                # create a mask to get unused model weights
                mask = np.ones(len(weights), dtype=bool)
                mask[models] = False
                # absent model aware weighted average
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * len(clustered_boxes) / (weighted_boxes[i, 2] + weights[mask].sum())
            elif conf_type == 'max':
                weighted_boxes[i, 1] = weighted_boxes[i, 1] / weights.max()
            elif not allows_overflow:
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * min(len(weights), len(clustered_boxes)) / weights.sum()
            else:
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * len(clustered_boxes) / weights.sum()
        overall_boxes.append(weighted_boxes)
    overall_boxes = np.concatenate(overall_boxes, axis=0)
    overall_boxes = overall_boxes[overall_boxes[:, 1].argsort()[::-1]]
    boxes = overall_boxes[:, 4:]
    scores = overall_boxes[:, 1]
    labels = overall_boxes[:, 0]
    return boxes, scores, labels

def weighted_boxes_fusion_tresh(boxes_list, scores_list, labels_list, weights=None, conf_tresh = 0.2, iou_thr=0.55, skip_box_thr=0.0, conf_type='avg', allows_overflow=False):
    boxes_fin = []
    scores_fin = []
    labels_fin = []
    
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights, iou_thr, skip_box_thr, conf_type, allows_overflow)
    
    if len(boxes) >0 :
        for score, box, label in zip(scores, boxes, labels):
            if score < conf_tresh:
                break

            boxes_fin.append(box)
            scores_fin.append(score)
            labels_fin.append(label)
    
    
    return np.array(boxes_fin), np.array(scores_fin), np.array(labels_fin)

In [ ]:
def RecoverCLAHE(sceneRadiance):
    clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(4, 4))
    for i in range(3):
        sceneRadiance[:, :, i] = clahe.apply((sceneRadiance[:, :, i]))

    return sceneRadiance

In [ ]:
def load_model(ckpt_path, conf=0.15, iou=0.30):
    model = torch.hub.load('/kaggle/input/yolo-lib',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload = True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 100  # maximum number of detections per image
    return model

In [ ]:
def predict(model, img, size = 1280, augment = False):
    bboxes = []
    scores = []
    classes = []
    results = model(img, size=size, augment = augment)  # custom inference size
    preds   = results.pandas().xyxy[0].values
    for bbox in preds:
        xmin = bbox[0]
        ymin = bbox[1]
        xmax = bbox[2]
        ymax = bbox[3]
        
        bboxes.append([xmin , ymin , xmax, ymax]) 
    
    scores = preds[: , 4]
    classes = preds[: , 5]
    return bboxes, scores, classes

In [ ]:
from norfair import Detection, Tracker

def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result


def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

In [ ]:
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)

frame_id = 0

In [ ]:
def show_prediction(img, bboxes):
    colors = [(0, 0, 255)]

    obj_names = ["s"]

    for box in bboxes:
        cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255,0,0), 2)
    
    img = Image.fromarray(img).resize((1280, 720))
    return img

In [ ]:
model_y5_1 = load_model(CKPT_PATH_1, CONF_1, IOU_1)
model_y5_2 = load_model(CKPT_PATH_2, CONF_2, IOU_2)
model_y5_3 = load_model(CKPT_PATH_3, CONF_3, IOU_3)

In [ ]:
df = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")

In [ ]:
from ast import literal_eval


def decode_annotations(annotaitons_str):
    """decode annotations in string to list of dict"""
    return literal_eval(annotaitons_str)

def load_image_with_annotations(img, annotaitons_str):
    annotations = decode_annotations(annotaitons_str)
    if len(annotations) > 0:
        for ann in annotations:
            cv2.rectangle(img, (ann['x'], ann['y']),
                (ann['x'] + ann['width'], ann['y'] + ann['height']),
                (255, 255, 255), thickness=2,)
            cv2.rectangle(img, (ann['x'], ann['y'] - 15),
                (ann['x'] + ann['width'], ann['y']),
                (255, 255, 255), -1 ,)
            cv2.putText(img, f'GT', (int(ann['x']), int(ann['y'] -3)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    return img

In [ ]:
dir = f'{DATASET_PATH}'
imgs = [dir + f for f in ('video_0/9674.jpg',
                          'video_0/20.jpg', 
                          'video_0/17.jpg', 
                          'video_0/100.jpg',
                          'video_0/246.jpg',
                          'video_0/337.jpg',
                          'video_0/358.jpg',
                          'video_0/1566.jpg',
                          'video_0/1854.jpg',
                          'video_0/1884.jpg',
                          'video_0/4224.jpg',
                          'video_0/4540.jpg',
                          'video_0/4582.jpg',
                          'video_0/4664.jpg',
                          'video_0/8215.jpg',
                          'video_0/8897.jpg',
                          'video_0/8928.jpg',
                          'video_0/9441.jpg',
                          'video_0/9859.jpg',
                          'video_0/12267.jpg',
                          'video_2/5748.jpg',
                          'video_2/5772.jpg',
                          'video_2/5820.jpg',
                          'video_2/5409.jpg',
                          'video_2/5482.jpg',
                          'video_2/5679.jpg',
                          'video_2/5712.jpg',
                          'video_2/5730.jpg',
                          'video_2/5751.jpg',
                          'video_2/5817.jpg',
                          'video_2/5868.jpg',
                          'video_2/6254.jpg',
                          'video_2/6339.jpg',
                          'video_2/10622.jpg',
                          'video_1/4159.jpg', 
                          'video_1/4183.jpg', 
                          'video_1/4501.jpg',
                          'video_1/5474.jpg',
                          'video_1/625.jpg',
                          'video_1/616.jpg',
                          'video_1/672.jpg',
                          'video_1/684.jpg',
                          'video_1/850.jpg',
                          'video_1/1927.jpg',
                          'video_1/2000.jpg',
                          'video_1/3903.jpg',
                          'video_1/3945.jpg',
                          'video_1/4051.jpg',
                          'video_1/4078.jpg',
                          'video_1/4096.jpg',
                          'video_1/4126.jpg',
                          'video_1/4456.jpg',
                          'video_1/4525.jpg',
                          'video_1/5267.jpg',
                          'video_1/5366.jpg',
                          'video_1/5411.jpg',
                          'video_1/5429.jpg',
                          'video_1/5492.jpg',
                          'video_1/5661.jpg',
                          'video_1/5892.jpg',
                          'video_1/6747.jpg',
                          'video_1/9082.jpg',)]

ids = ids = ['0-9674', '0-20', '0-17', '0-100', '0-246', '0-337', '0-358', '0-1566',
             '0-1854', '0-1884', '0-4224', '0-4540', '0-4582', '0-4664', '0-8215',
             '0-8897', '0-8928', '0-9441', '0-9859', '0-12267',
             '2-5748', '2-5772', '2-5820', '2-5409', '2-5482',
             '2-5679', '2-5712', '2-5730', '2-5751',  '2-5817',
             '2-5868', '2-6254', '2-6339', '2-10622',
             '1-4159', '1-4183', '1-4501', '1-5474', '1-625', '1-616', '1-672', '1-684',
             '1-850', '1-1927', '1-2000', '1-3903', '1-3945',
             '1-4051', '1-4078', '1-4096', '1-4126', '1-4456',
             '1-4525', '1-5267', '1-5366', '1-5411', '1-5429',
             '1-5492', '1-5661', '1-5892',  '1-6747', '1-9082',]


# for img, idx in zip(imgs, ids):
#     im = cv2.imread(img)
#     #im = RecoverCLAHE(im)
#     im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
#     bboxes, scores, classes = predict(model_y5_1, im, size = IMG_SIZE_1, augment = True)
#     annot = df.query("image_id == @idx").annotations.item()
#     im = load_image_with_annotations(im, annot)
#     display(show_prediction(im, bboxes))

## ENSEMBLE


In [ ]:
def final_pred(img_in, w_conf = 0.1, w_nms = 0.3):
        bboxes_y5_1 = []
        scores_y5_1 = []
        classes_y5_1 = []
        
        bboxes_y5_2 = []
        scores_y5_2 = []
        classes_y5_2 = []
        
        bboxes_y5_3 = []
        scores_y5_3 = []
        classes_y5_3 = []
        
        bboxes_fin = [] 
        scores_fin = [] 
        bbclasses_fin = []

        # predykcja Y5
        bboxes_y5_1, scores_y5_1, classes_y5_1 = predict(model_y5_1, img_in, size = IMG_SIZE_1, augment = False)
        bboxes_y5_1 = np.array(bboxes_y5_1)
        
        if DEBUG:
            print("*********************")
            print("BBOXES MODEL - 1")
            print(bboxes_y5_1)
            print("SCORES")
            print(scores_y5_1)
            print("\n")
        
        if len(bboxes_y5_1)>0:
            bboxes_y5_1[:, [0, 2]] /= 1280
            bboxes_y5_1[:, [1, 3]] /= 720
        
        
        
        bboxes_y5_2, scores_y5_2, classes_y5_2 = predict(model_y5_2, img_in, size = IMG_SIZE_2, augment = False)
        bboxes_y5_2 = np.array(bboxes_y5_2)
        
        if DEBUG:
            print("BBOXES MODEL - 2")
            print(bboxes_y5_2)
            print("SCORES")
            print(scores_y5_2)
            print("\n")
        
        if len(bboxes_y5_2)>0:
            bboxes_y5_2[:, [0, 2]] /= 1280
            bboxes_y5_2[:, [1, 3]] /= 720
            
        
        bboxes_y5_3, scores_y5_3, classes_y5_3 = predict(model_y5_3, img_in, size = IMG_SIZE_3, augment = False)
        bboxes_y5_3 = np.array(bboxes_y5_3)
        
        if DEBUG:
            print("*********************")
            print("BBOXES MODEL - 1")
            print(bboxes_y5_3)
            print("SCORES")
            print(scores_y5_3)
            print("\n")
        
        if len(bboxes_y5_3)>0:
            bboxes_y5_3[:, [0, 2]] /= 1280
            bboxes_y5_3[:, [1, 3]] /= 720

        
        
        bboxes_fin, scores_fin, bbclasses_fin = weighted_boxes_fusion_tresh([bboxes_y5_1, bboxes_y5_2, bboxes_y5_3],
                                                                      [scores_y5_1, scores_y5_2, scores_y5_3],
                                                                      [classes_y5_1, classes_y5_2, classes_y5_3],
                                                                      weights=[1,1, 1],
                                                                      conf_tresh = 0.1,
                                                                      iou_thr= 0.25,
                                                                      skip_box_thr=0.01,
                                                                      conf_type='avg')

        #print(bboxes_fin.shape)
        if len(bboxes_fin) > 0:
            bboxes_fin[:, [0, 2]] *= 1280
            bboxes_fin[:, [1, 3]] *= 720
        else:
            bboxes_fin = bboxes_fin.reshape(0,4)
        
        if DEBUG:
            print("BBOXES FINAL")
            print(bboxes_fin)
            print("SCORES")
            print(scores_fin)
            print("\n\n")

        return bboxes_fin, scores_fin, bbclasses_fin


In [ ]:
DEBUG = True
dir = f'{DATASET_PATH}'
imgs = [dir + f for f in ('video_0/9674.jpg',
                          'video_0/20.jpg', 
                          'video_0/17.jpg', 
                          'video_0/100.jpg',
                          'video_0/246.jpg',
                          'video_0/337.jpg',
                          'video_0/358.jpg',
                          'video_0/1566.jpg',
                          'video_0/1854.jpg',
                          'video_0/1884.jpg',
                          'video_0/4224.jpg',
                          'video_0/4540.jpg',
                          'video_0/4582.jpg',
                          'video_0/4664.jpg',
                          'video_0/8215.jpg',
                          'video_0/8897.jpg',
                          'video_0/8928.jpg',
                          'video_0/9441.jpg',
                          'video_0/9859.jpg',
                          'video_0/12267.jpg',
                          'video_2/5748.jpg',
                          'video_2/5772.jpg',
                          'video_2/5820.jpg',
                          'video_1/4159.jpg', 
                          'video_1/4183.jpg', 
                          'video_1/4501.jpg',
                          'video_1/5474.jpg',
                          'video_1/625.jpg',
                          'video_1/850.jpg',
                          'video_1/1927.jpg',
                          'video_1/2000.jpg',
                          'video_1/3903.jpg',
                          'video_1/3945.jpg',
                          'video_1/4051.jpg',
                          'video_1/4078.jpg',
                          'video_1/4096.jpg',
                          'video_1/4126.jpg',
                          'video_1/4456.jpg',
                          'video_1/4525.jpg',
                          'video_1/5267.jpg',
                          'video_1/5366.jpg',
                          'video_1/5411.jpg',
                          'video_1/5429.jpg',
                          'video_1/5492.jpg',
                          'video_1/5661.jpg',
                          'video_1/5892.jpg',
                          'video_1/6747.jpg',
                          'video_1/9082.jpg',)]

ids = ids = ['0-9674', '0-20', '0-17', '0-100', '0-246', '0-337', '0-358', '0-1566',
             '0-1854', '0-1884', '0-4224', '0-4540', '0-4582', '0-4664', '0-8215',
             '0-8897', '0-8928', '0-9441', '0-9859', '0-12267',
             '2-5748', '2-5772', '2-5820',
             '1-4159', '1-4183', '1-4501', '1-5474', '1-625',
             '1-850', '1-1927', '1-2000', '1-3903', '1-3945',
             '1-4051', '1-4078', '1-4096', '1-4126', '1-4456',
             '1-4525', '1-5267', '1-5366', '1-5411', '1-5429',
             '1-5492', '1-5661', '1-5892',  '1-6747', '1-9082',]


for img, idx in zip(imgs, ids):
    im = cv2.imread(img)
    imy = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    print(f"{img} -> {idx}")
    bboxes, scores, _ = final_pred(imy, w_conf = 0.2, w_nms = 0.1)
    annot = df.query("image_id == @idx").annotations.item()
    imy = load_image_with_annotations(imy, annot)
    display(show_prediction(imy, bboxes))
    

In [ ]:
%cd /kaggle/working/

In [ ]:
DEBUG = False

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
#model = load_model(CKPT_PATH, conf=CONF, iou=IOU)

for (image_np, sample_prediction_df) in iter_test:
    
    #bboxes, scores = predict(model, image_np, size = IMG_SIZE, augment = True)
    
    bboxes, scores, _ = final_pred(image_np, w_conf = 0.4, w_nms = 0.1)
    
    predictions = []
    detects = []
    
    for i in range(len(bboxes)):
        box = bboxes[i]
        score = scores[i]
        x_min = int(box[0])
        y_min = int(box[1])
        x_max = int(box[2])
        y_max = int(box[3])
        
        bbox_width = x_max - x_min
        bbox_height = y_max - y_min
        detects.append([x_min, y_min, x_max, y_max, score])
        
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
            
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]

        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)
    frame_id += 1

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()